## Finding a Regression Model to Predict Weekly Natural Gas Prices

This notebook details the exploratory steps to develop a predictive model that investigates and utilizes prior natural gas prices, natural gas storage data, and Global Forecast System Ensemble temperature predictions (for forecast ranges of 1-16 days at the 850 hPa level).
    
---

*Initial Findings:*
 - Gradient boosted regression appears to perform the best of the regressors tested (Ridge, Lasso, ElasticNet, SVR, Random Forest, Gradient Boosting).
 - Linear regressors become strongly focused on the day1prior price, which is understandable, but they fail to utilize much price sensitivity to upcoming temperature anomalies
 - On the other hand, gradient boosting distributes feature importance much more, and its scoring is improved over all other models tested

xlrd is needed to support pandas import of xls files
netcdf4 necessary to import GEFS data, which are generously formatted by the NOAA Earth Systems Research Lab 
at https://www.esrl.noaa.gov/psd/forecasts/reforecast2/download.html

In [14]:
import pandas as pd
import numpy as np
import xlrd
import matplotlib.pyplot as plt
import netCDF4 as nc

### Importing NatGas data
---

Read historical gas price (daily & weekly frequency) and storage information downloaded from
- https://www.eia.gov/dnav/ng/ng_pri_fut_s1_d.htm
- https://www.eia.gov/dnav/ng/ng_pri_fut_s1_w.htm
- https://www.eia.gov/dnav/ng/ng_stor_wkly_s1_w.htm

In [50]:
gasPrices = pd.read_excel('./data/NG_PRI_FUT_S1_D.xls',sheetname='Data 1')
gasPricesWeek = pd.read_excel('./data/NG_PRI_FUT_S1_W.xls',sheetname='Data 1')
gasStorage = pd.read_excel('./data/NG_STOR_WKLY_S1_W.xls',sheetname='Data 1')

Clean these dataframes by dropping rows with metadata, reseting indices, and re-naming column labels.
Also convert price column to float.

In [51]:
gasPrices = gasPrices.drop([0,1]).rename( columns={'Back to Contents':'Date','Data 1: Spot Price':'Price'})
gasPrices['Price'] = gasPrices['Price'].astype(float)
gasPrices.reset_index(drop=True,inplace=True)

gasPricesWeek = gasPricesWeek.drop([0,1]).rename( columns={'Back to Contents':'Date','Data 1: Spot Price':'Price_WeekAvg'})
gasPricesWeek['Price_WeekAvg'] = gasPricesWeek['Price_WeekAvg'].astype(float)
gasPricesWeek.reset_index(drop=True,inplace=True)

gasStorage = gasStorage.drop([0,1]).drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7', \
                            'Unnamed: 8'],axis=1).rename(columns={'Back to Contents':'Date'})
gasStorage = gasStorage.rename(columns = {'Data 1: Weekly Working Gas in Underground Storage': 'gasStorage'})
gasStorage.reset_index(drop=True,inplace=True)

Re-format Date column so that it can be merged with the temperature data we're about to import

In [52]:
gasPrices['Date'] = pd.to_datetime(gasPrices['Date'].dt.strftime('%Y-%m-%d'))
gasPricesWeek['Date'] = pd.to_datetime(gasPricesWeek['Date'].dt.strftime('%Y-%m-%d'))
gasStorage['Date'] = pd.to_datetime(gasStorage['Date'].dt.strftime('%Y-%m-%d'))

### Reading GEFS forecast temperature data
---

Read downloaded netcdf data from NOAA/ESRL. This is the Day1-8 file (a separate Day9-16 file will be imported shortly)

In [15]:
file = './data/tmp_pres_latlon_mean_20100101_20191231_joeyfVioTi.nc'
data = nc.Dataset(file,mode='r')

Print the variables within this nc file. We're mainly interested in Temperature and intTime as these will be used in our dataframe. Pressure is nothing but '850' because we requested the data only at the 850 level. We also don't really care about lat/lon because we're simply taking a mean of the entire domain for this example.

In [20]:
for idx, var in enumerate(data.variables):
    print(f'Variable {idx+1}: {var}')

Variable 1: time
Variable 2: intTime
Variable 3: lat
Variable 4: lon
Variable 5: fhour
Variable 6: intValidTime
Variable 7: pressure
Variable 8: Temperature


Create empty arrays to hold the model run times and also the forecast temperature data for each day.
Loop through each of the forecast times with the outer loop. Append that forecast time to the modelRunDay list, which we will use as our Date column to merge with NatGas data. The inner loop iterates through the forecast valid times that we want -- [4,12,20,26,...,42] are the positions of 12Z valid times for D1 through D8.

In [23]:
modelRunDay, week1 = [], [[],[],[],[],[],[],[],[]]

for i in range(len(data.variables['intTime'][:])):
    
    modelRunDay.append(data.variables['intTime'][i])
    
    for idx, time in enumerate([4,12,20,26,30,34,38,42]):
        
        week1[idx].append(np.mean(data.variables['Temperature'][i][time]))
        

Convert the arrays into two dataframes and join them together

In [28]:
tempH85 = pd.DataFrame(week1).transpose().rename(columns={0:'D1',1:'D2',2:'D3',3:'D4',4:'D5',5:'D6',6:'D7',7:'D8'})
runDate = pd.DataFrame(modelRunDay, columns=['RunDate'])

week1data = runDate.join(tempH85)

Here's a look at the dataframe for "Week 1" -- note that temperature is in Kelvin, but no need to convert. Kelvin works for now and we'll scale the data in pre-processing prior to feeding it to our various regressors.

In [30]:
week1data.head(10)

,RunDate,D1,D2,D3,D4,D5,D6,D7,D8
0,2010010100,271.684,268.322,268.512,269.092,268.85,268.8,268.317,268.118
1,2010010200,268.152,268.244,269.368,269.375,269.219,268.166,267.612,268.127
2,2010010300,267.909,269.041,269.242,269.461,268.22,267.155,266.474,267.185
3,2010010400,268.845,269.118,269.2,268.011,266.414,266.214,267.794,269.339
4,2010010500,268.844,269.123,268.548,267.311,267.009,268.89,269.888,270.21
5,2010010600,269.236,268.874,267.697,266.899,268.713,269.772,269.966,271.612
6,2010010700,269.143,268.041,267.285,268.91,269.864,269.711,271.664,274.069
7,2010010800,268.048,267.391,269.361,270.418,270.94,273.49,275.526,275.384
8,2010010900,267.286,269.224,270.189,271.224,274.246,275.433,275.199,274.869
9,2010011000,269.171,269.707,270.809,275.053,276.126,275.37,274.347,274.406


#### Reading data for "week 2" i.e. D9-16 temp data
Note that we've changed the positional indices in the inner loop to match 12Z for D9-16. 

In [33]:
file = './data/tmp_pres_latlon_mean_20100101_20191231_joeyfVioTi_t190.nc'
data = nc.Dataset(file,mode='r')

modelRunDay, week2 = [], [[],[],[],[],[],[],[],[]]

for i in range(len(data.variables['intTime'][:])):
    
    modelRunDay.append(data.variables['intTime'][i])
    
    for idx, time in enumerate([3,7,11,15,19,23,27,31]):
        
        week2[idx].append(np.mean(data.variables['Temperature'][i][time]))
        
tempH85 = pd.DataFrame(week2).transpose().rename(columns={0:'D9',1:'D10',2:'D11',3:'D12',
                                                          4:'D13',5:'D14',6:'D15',7:'D16'})
runDate = pd.DataFrame(modelRunDay, columns=['RunDate'])

week2data = runDate.join(tempH85)

Just to make sure everything looks OK, here's some week2 data.

In [34]:
week2data.head(10)

,RunDate,D9,D10,D11,D12,D13,D14,D15,D16
0,2010010100,268.419,268.76,269.685,269.67,270.269,270.079,270.129,271.228
1,2010010200,270.101,271.26,271.354,271.467,272.021,271.985,271.618,271.63
2,2010010300,268.929,270.522,271.462,272.581,273.227,273.034,272.112,271.353
3,2010010400,270.699,271.57,272.032,272.029,272.127,272.516,272.106,272.298
4,2010010500,271.412,272.904,274.181,274.567,274.529,273.659,273.305,274.13
5,2010010600,273.521,274.318,274.573,274.444,273.172,271.933,271.817,271.661
6,2010010700,274.878,275.013,274.389,273.035,272.088,272.088,272.553,273.142
7,2010010800,274.883,273.94,272.321,271.791,272.25,273.083,273.897,274.448
8,2010010900,274.847,273.509,271.989,272.195,273.647,275.087,275.748,275.646
9,2010011000,274.416,275.427,276.986,278.519,279.588,279.671,278.725,276.574


Merging our weekly dataframs together to form our "master" temp dataframe

In [35]:
masterTempData = week1data.merge(week2data, on='RunDate', how='left')
masterTempData['Date'] = pd.to_datetime(masterTempData['RunDate'].astype(str),format='%Y%m%d%H')

### Merge temp and natgas data & prep rows for modeling (e.g., set up lags)
___

We drop 'RunDate' since it's no longer needed and also merge our weekly prices (which will eventually form our target).

In [40]:
everything = masterTempData.merge(gasPrices, on='Date',how='left').drop(['RunDate'],axis=1)
everything = everything.merge(gasPricesWeek,on='Date',how='left')

Start creating our "lag" columns. The last several days (and perhaps more) of 850 temps may have some impact on upcoming natgas prices so let's appropriately shift the D1 column to make prior day features. 

---
*Note: we are not using observations for prior days. This might be more appropriate, but D1 12-hour ensemble means should be fairly accurate on the scale we need. Given we've already found/loaded the data, let's just use that.*

In [41]:
everything['dayPrior1'] = everything['D1'].shift(1)
everything['dayPrior2'] = everything['D1'].shift(2)
everything['dayPrior3'] = everything['D1'].shift(3)
everything['dayPrior4'] = everything['D1'].shift(4)
everything['dayPrior5'] = everything['D1'].shift(5)

We also shift our natgas prices to make new lagged columns. Obviously, next week's average price should be pretty sensitive to recent prices so we want to include those. The number of lags we use is arbitrary of course.

In [42]:
everything['priceLag1'] = everything['Price'].shift(1)
everything['priceLag2'] = everything['Price'].shift(2)
everything['priceLag3'] = everything['Price'].shift(3)

Shift our weekly prices up by 7 (1 week) and 14 rows (2 weeks) to form our targets.

In [44]:
everything['weekAvgPriceLag-1'] = everything['Price_WeekAvg'].shift(-7)
everything['weekAvgPriceLag-2'] = everything['Price_WeekAvg'].shift(-14)

Let's also create some trend columns for the hell of it.

In [45]:
# We are assuming we won't have the current day's closing price -- thus we use the first lag and second lag for the
# one day trend and first lag minus third lag for the two-day trend
everything['oneDayTrend'] = everything['priceLag1'] - everything['priceLag2']
everything['twoDayTrend'] = everything['priceLag1'] - everything['priceLag3']

Merge gas storage data with our main dataframe as well. Create a couple more lag columns too.

In [55]:
everythingPlusStorage = everything.merge(gasStorage,on='Date',how='left')

everythingPlusStorage['storageLag1'] = everythingPlusStorage['gasStorage'].shift(7)
everythingPlusStorage['storageLag2'] = everythingPlusStorage['gasStorage'].shift(14)

This cleaning step is necessary to remove rows that have "--" due to missing model forecast data.

In [56]:
everythingPlusStorageCleaned = everythingPlusStorage[(everythingPlusStorage['D9'] > 0) \
                                                    & (everythingPlusStorage['D1'] > 0)
                                                    & (everythingPlusStorage['dayPrior1'] > 0) \
                                                    & (everythingPlusStorage['dayPrior2'] > 0) \
                                                    & (everythingPlusStorage['dayPrior3'] > 0) \
                                                    & (everythingPlusStorage['dayPrior4'] > 0) \
                                                    & (everythingPlusStorage['dayPrior5'] > 0)]

Create two dataframes that we'll use for our models. The plan is to make a model for next week's prediction, and then a model for the two week prediction. That could change in the future but we'll leave it at that for now.

In [59]:
week1prediction = everythingPlusStorageCleaned.drop(['weekAvgPriceLag-2','Price_WeekAvg',\
                                  'gasStorage'],axis=1)
week2prediction = everythingPlusStorageCleaned.drop(['weekAvgPriceLag-1','Price_WeekAvg',\
                                  'gasStorage'],axis=1)

week1prediction = week1prediction.dropna(how='any')
week2prediction = week2prediction.dropna(how='any')

## Develop Modeling
___

Note that we'll use a combo of some regression models with gridsearchcv and some model classes with a built-in cv. Why? Partially because I was just dinking around it with initially, but it's handy to get a feel for both ways. According to sklearn's documentation, classes with built-in cv may perform a little better than their straight-up counterparts at times: <br>
*The advantage of using a cross-validation estimator over the canonical Estimator class along with grid search is that they can take advantage of warm-starting by reusing precomputed results in the previous steps of the cross-validation process*

In [62]:
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, LassoCV, ElasticNetCV 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

Make a quick function to print mean squared error and the coefficient of determination when it comes time to check our models against the test sets.

In [63]:
def model_scorer(target, prediction):
    
    print(f'The mean squared error is: {mean_squared_error(target,prediction)}')
    print(f'The coefficient of determination (r2) is: {r2_score(target,prediction)}')

Create our regressors and our target variable to begin development for week 1 prediction. For week 1, we will drop from X...
- Date: At this time, we'll leave it out. Perhaps we could use the week or month down the line.
- Price: We won't have the closing price for the current day when we run our model. So we gotta kick it to the curb.
- weekAvgPriceLag-1: This is our target variable so we have to drop this from X.

For Y, we set it to our target: weekAvgPriceLag-1.

In [64]:
X = week1prediction.drop(['Date','Price','weekAvgPriceLag-1'],axis=1)
Y = week1prediction['weekAvgPriceLag-1'].values

Split our features and targets into training and test data, with an 80/20 split.

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

Create our scaler object, then use fit_transform to fit the scaler and transform X_train & X_test. We transform the data by representing them as standardized anomalies (i.e., subtract the mean and divide by the standard deviation). Many ML models perform better if we scale the features in such a manner.

In [67]:
scaler = StandardScaler()

train_scaled = scaler.fit_transform(X_train)
test_scaled = scaler.transform(X_test)

Define a quick function to print the attributes of the best estimator when we perform cross validation.

In [83]:
def printCVbest(modelObject):
    print(f'Best estimator: {modelObject.best_estimator_}')
    print(f'Best parameters: {modelObject.best_params_}')
    print(f'Best score: {modelObject.best_score_}')

### Linear Regressors
___

#### Ridge Regression

Create our ridge regression oject and a list of alpha parameters. Ridge uses least squares regression, but with a normalization of alpha*||w||^2_2 -- alpha regulates the amount of regularization that occurs.

Perform a grid search against the supplied alpha parameters to find the best scoring alpha. Also perform this on k-fold of 5. Lastly, we fit the regressor object to our data.

In [85]:
ridge = Ridge(solver='auto')
parameters = {'alpha':[1e-2,1,5,10,20,40]}
ridgeRegressor = GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
ridgeRegressor.fit(train_scaled,y_train)

printCVbest(ridgeRegressor)

Best estimator: Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
Best parameters: {'alpha': 10}
Best score: -0.03891558775657337


Use the best estimator to predict next week's average natgas price and use our model_scorer function to print the mean squared error and coefficient of determination.

In [91]:
bestRidge = ridgeRegressor.best_estimator_
predictionRidge = bestRidge.predict(test_scaled)

model_scorer(y_test,predictionRidge)

The mean squared error is: 0.1534526489299111
The coefficient of determination (r2) is: 0.8072589128074912


#### Lasso Regression
Lasso regression uses a different regularization: alpha*||w||. With this change, minimization of the loss function is useful for feature selection and avoiding overfitting, as it drives many coefficients to 0.  

In [90]:
lassoRegressor = LassoCV(alphas=[1e-2,1,5,10,20,40],cv=5)
lassoRegressor.fit(train_scaled,y_train)

LassoCV(alphas=[0.01, 1, 5, 10, 20, 40], copy_X=True, cv=5, eps=0.001,
    fit_intercept=True, max_iter=1000, n_alphas=100, n_jobs=1,
    normalize=False, positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

In [92]:
predictionLasso = lassoRegressor.predict(test_scaled)
model_scorer(y_test,predictionLasso)

The mean squared error is: 0.15144733172193925
The coefficient of determination (r2) is: 0.8097776508125085


#### ElasticNet Regression
ElasticNet regression combines the regularizers from Ridge and Lasso regression. The parameter uses a ratio parameter to achieve this hybrid function we wish to minimize.

In [94]:
elasticRegressor = ElasticNetCV(alphas=[1e-2,1,5,10,20,40],l1_ratio=[.1, .5, .7, .9, .95, .99, 1],cv=5)
elasticRegressor.fit(train_scaled,y_train)

ElasticNetCV(alphas=[0.01, 1, 5, 10, 20, 40], copy_X=True, cv=5, eps=0.001,
       fit_intercept=True, l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
       max_iter=1000, n_alphas=100, n_jobs=1, normalize=False,
       positive=False, precompute='auto', random_state=None,
       selection='cyclic', tol=0.0001, verbose=0)

In [95]:
predictionElastic = elasticRegressor.predict(test_scaled)
model_scorer(y_test,predictionElastic)

The mean squared error is: 0.15144733172193925
The coefficient of determination (r2) is: 0.8097776508125085


Notice that the scores for Lasso and ElasticNet are the same. If we print out the coefficients from the models chosen by CV for each of these regressions, we notice they're indeed the same. ElasticNetCV settled on an l1_ratio of 1, which means it's essentially Lasso at this point. So for linear regression, large importance is placed on the prior day's natgas price, and not much else. As you'll see, we can do better.

In [101]:
elasticRegressor.coef_, lassoRegressor.coef_, elasticRegressor.l1_ratio_

(array([ 0.        , -0.        , -0.        , -0.        , -0.        ,
        -0.        , -0.        , -0.        , -0.        , -0.        ,
        -0.        , -0.        , -0.        , -0.        , -0.        ,
        -0.        ,  0.        ,  0.        , -0.        , -0.        ,
        -0.        ,  0.79218602,  0.        ,  0.0180529 ,  0.        ,
        -0.        , -0.        , -0.        ]),
 array([ 0.        , -0.        , -0.        , -0.        , -0.        ,
        -0.        , -0.        , -0.        , -0.        , -0.        ,
        -0.        , -0.        , -0.        , -0.        , -0.        ,
        -0.        ,  0.        ,  0.        , -0.        , -0.        ,
        -0.        ,  0.79218602,  0.        ,  0.0180529 ,  0.        ,
        -0.        , -0.        , -0.        ]),
 1.0)

### Ensemble Regression
___

In [105]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

#### Random Forest

This model fits a number of decision trees on various sub-sets of the data, thereby creating an ensemble of trees. The decision is then averaged from the trees.

---
Here we'll use cross validation to test several n_estimators in our regressor.

In [103]:
n_estimators = [5,10,25,50,75,100,150,200,300,400]
param_grid = dict(n_estimators=n_estimators)

In [106]:
rfRegressor = RandomForestRegressor()
gridSearched = GridSearchCV(rfRegressor,param_grid,scoring='neg_mean_squared_error',cv=5)
gridSearched.fit(train_scaled,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10, 25, 50, 75, 100, 150, 200, 300, 400]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

We can print out the importance of features in this ensemble. For this random forest, it acts very similar to the linear regressors, in that it places almost all importance on the prior days' prices.

In [113]:
# Checking the importance of each feature in the random forest model
for i in range(len(X.columns)):
    print(f'{X.columns[i]}: {gridSearched.best_estimator_.feature_importances_[i]}')

D1: 0.000863858801516324
D2: 0.001001498337021604
D3: 0.0017828628032362458
D4: 0.001061301466399699
D5: 0.0016337496784022943
D6: 0.0019637870428739247
D7: 0.0012144638439771636
D8: 0.0009524505041163913
D9: 0.0008186282748865373
D10: 0.001061967021934457
D11: 0.0014605286512548137
D12: 0.0009148201009567135
D13: 0.0013758124885811436
D14: 0.0007954145747766344
D15: 0.0006800143227425303
D16: 0.0010446697300350773
dayPrior1: 0.0011016768552147723
dayPrior2: 0.0007145163299351637
dayPrior3: 0.0008605261796679624
dayPrior4: 0.0011093132447349773
dayPrior5: 0.0012227562977073397
priceLag1: 0.5016168416362438
priceLag2: 0.3488349121983397
priceLag3: 0.1150893942460285
oneDayTrend: 0.00209740440315335
twoDayTrend: 0.0023013904634145836
storageLag1: 0.0039365086125947414
storageLag2: 0.002488931890253549


In [108]:
predictionRF = gridSearched.predict(test_scaled)
model_scorer(y_test,predictionRF)

The mean squared error is: 0.15905823187857968
The coefficient of determination (r2) is: 0.8002181340434331


#### Gradient Boosted Regressor

Gradient boosting builds trees as it iterates forward and fits a tree with each step (controlled by the learning rate) along the loss function. 

In [114]:
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
n_estimators = [5,10,25,50,75,100,150,200,300,400]
param_grid = dict(learning_rate=learning_rate,n_estimators=n_estimators)

In [115]:
gbRegressor = GradientBoostingRegressor()
gridSearched = GridSearchCV(gbRegressor,param_grid,scoring='neg_mean_squared_error',cv=5)
gridSearched.fit(train_scaled,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3], 'n_estimators': [5, 10, 25, 50, 75, 100, 150, 200, 300, 400]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

Once again, we print out the importance placed on each feature. Except this time, we notice temperature features are at least an order of magnitude higher in most cases. While most importance is still placed on the prior days' prices, this at least offers a notable change from the prior models. And our scoring suggests gradient boosting may perform the best out of the models tested.

In [117]:
# Checking the importance of each feature in the random forest model
for i in range(len(X.columns)):
    print(f'{X.columns[i]}: {gridSearched.best_estimator_.feature_importances_[i]}')

D1: 0.011588540643157659
D2: 0.0034521042772727908
D3: 0.0034426331127390313
D4: 0.00675513456833057
D5: 0.04544268373917919
D6: 0.02629212934265126
D7: 0.006220530465221269
D8: 0.0
D9: 0.014181771222104449
D10: 0.007016496362222521
D11: 0.02195806351811663
D12: 0.012341521514808208
D13: 0.019353143082012053
D14: 0.012435141120623848
D15: 0.0014244280476269952
D16: 0.010045034104553445
dayPrior1: 0.01913988280701525
dayPrior2: 0.007613982015253091
dayPrior3: 0.02723741343850959
dayPrior4: 0.003092123867839528
dayPrior5: 0.02535576005872475
priceLag1: 0.33192338784027975
priceLag2: 0.21612688221022086
priceLag3: 0.07080436634334246
oneDayTrend: 0.010196915622783485
twoDayTrend: 0.03278046827078007
storageLag1: 0.014970674970432953
storageLag2: 0.038808787434198265


In [116]:
predictionGB = gridSearched.predict(test_scaled)
model_scorer(y_test,predictionGB)

The mean squared error is: 0.12013317380053642
The coefficient of determination (r2) is: 0.8491091637213914


Use the joblib module to save our Gradient Boosted model to a file, which we can then load in the future via 'load'.

In [119]:
from joblib import dump, load

In [120]:
dump(gridSearched.best_estimator_, 'gb_8jan2019.joblib')

['gb_8jan2019.joblib']